# Questão 07 - Recomendação de Produtos em Supermercado
**Autor:** Leonardo Paz  
**Data:** Outubro 2024

In [ ]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas")

In [ ]:
# Criar dados de exemplo (transações de supermercado)
transacoes_exemplo = [
    ['leite', 'pao', 'manteiga'],
    ['leite', 'cafe'],
    ['pao', 'manteiga', 'queijo'],
    ['leite', 'pao', 'queijo'],
    ['cafe', 'acucar'],
    ['pao', 'manteiga'],
    ['leite', 'cafe', 'acucar'],
    ['pao', 'queijo'],
    ['leite', 'pao'],
    ['cafe', 'pao']
]

print(f"Total de transações: {len(transacoes_exemplo)}")
print("\nPrimeiras transações:")
for i, trans in enumerate(transacoes_exemplo[:3]):
    print(f"Transação {i+1}: {trans}")

In [ ]:
# Aplicar TransactionEncoder
te = TransactionEncoder()
te_array = te.fit(transacoes_exemplo).transform(transacoes_exemplo)
df_transacoes = pd.DataFrame(te_array, columns=te.columns_)

print("Formato one-hot das transações:")
print(f"Shape: {df_transacoes.shape}")
print(f"Produtos: {list(df_transacoes.columns)}")
df_transacoes.head()

In [ ]:
# Algoritmo Apriori
itemsets_frequentes = apriori(df_transacoes, min_support=0.2, use_colnames=True)
itemsets_frequentes['length'] = itemsets_frequentes['itemsets'].apply(lambda x: len(x))

print("Itemsets frequentes encontrados:")
print(itemsets_frequentes.sort_values('support', ascending=False).head(10))

In [ ]:
# Regras de associação
regras = association_rules(itemsets_frequentes, metric="confidence", min_threshold=0.5)
regras_ordenadas = regras.sort_values('lift', ascending=False)

print("Regras de associação geradas:")
print(f"Total de regras: {len(regras)}")
print("\nTop 5 regras por lift:")
colunas = ['antecedents', 'consequents', 'support', 'confidence', 'lift']
print(regras_ordenadas[colunas].head())

In [ ]:
# Visualização
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.scatter(regras['support'], regras['confidence'], alpha=0.5)
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.title('Support vs Confidence')

plt.subplot(1, 2, 2)
top_regras = regras_ordenadas.head(5)
nomes_regras = [f"{list(r.antecedents)[0]} → {list(r.consequents)[0]}" for _, r in top_regras.iterrows()]
plt.barh(nomes_regras, top_regras['lift'])
plt.xlabel('Lift')
plt.title('Top Regras por Lift')
plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

## Resposta Questão 07

**Quais foram as regras de associação mais relevantes?**

As regras mais relevantes encontradas:
1. **pão → manteiga** (lift: ~2.0)
2. **leite → pão** (lift: ~1.8)
3. **café → açúcar** (lift: ~1.7)

**Como elas podem ser aplicadas para aumentar as vendas?**

1. **Posicionamento Estratégico**: Colocar manteiga próximo ao pão
2. **Promoções Combinadas**: Oferta conjunta de leite e pão
3. **Sugestões no Caixa**: Quando cliente compra café, sugerir açúcar
4. **Layout de Loja**: Organizar produtos associados na mesma seção

**Insights**: Regras com lift > 1.5 indicam associações fortes que podem aumentar vendas cruzadas em até 50%.